In [1]:
import chromadb
client = chromadb.PersistentClient()

In [2]:
client.delete_collection("answers")

In [3]:
answers = client.create_collection(
    name="answers"
)

In [5]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel("../resources/comfort_datasets.xlsx")
df

,user,answer,class
0,12시 땡!,오늘 하루도 수고했어요 좋은 꿈 꾸세요,일상
1,1지망 학교 떨어졌어,2지망학교는 꼭 붙을거예요!!,일상
2,3박4일 놀러가고 싶다,요즘은 호캉스가 유행이래요! 호텔에서 맛있는것도 먹고 수영도 하고 영화도 보면 너무...,일상
3,3박4일 정도 놀러가고 싶다,요즘은 호캉스가 유행이래요! 호텔에서 맛있는것도 먹고 수영도 하고 영화도 보면 너무...,일상
4,5분 안에 다시 올게,네. 혼자 쓸쓸히 티비나 보면서(흑흑) 기다릴게요!,일상
...,...,...,...
34544,내가 듣고싶어하는 말:자살하지마세요.제발요.. 또는... 죽지 마세요..,"힘내세요, 선생님. 울어도 되고, 쉬어도 돼요. 그렇지만 언젠가는 꼭 다시 일어나 ...",사연
34545,그냥 다 놓고 사라져버리고 싶다,힘내세요 .....! 저도 가끔 이세상에서 없어지고 싶다는 생각을 할 때가 종종 있...,사연
34546,인간의 추악한 모습을 많이봐서 너무 많이실망했다 내가 그들과 같은 인간이라는게 싫다,그들과 같은 인간이지만 그들과 다르게 문제점을 파악하고 있으니 다른 인간 인거에요!...,사연
34547,죽거나 전학가고싶어요.. 제발 도와주세요..,얼마나 힘들고 괴로우면 죽고싶다는 생각을 할까요..짐작도 못할 만큼 힘들겠지만.. ...,사연


In [6]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer
import torch
import numpy as np
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
sess = InferenceSession("../models/onnx/sbert-model_uint8.onnx" , providers=["CPUExecutionProvider"])

/opt/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def mean_pooling(model_output, attention_mask):
    model_output = torch.from_numpy(model_output[0])
    # First element of model_output contains all token embeddings
    token_embeddings = model_output
    attention_mask = torch.from_numpy(attention_mask)
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask, input_mask_expanded, sum_mask

In [8]:
def embedding_query(query: str, normalize_embeddings=False) -> np.ndarray:
    # user turn sequence to query embedding
    model_inputs = tokenizer(query, return_tensors="pt")
    inputs_onnx = {k: v.cpu().detach().numpy()
                   for k, v in model_inputs.items()}
    sequence = sess.run(None, inputs_onnx)
    query_embedding = mean_pooling(
        sequence, inputs_onnx["attention_mask"])[0][0]

    if normalize_embeddings:
        query_embedding = query_embedding / \
            np.linalg.norm(query_embedding)

    return query_embedding.numpy()

In [9]:
ids = []
metadatas = []
embeddings = [embedding_query(sen, normalize_embeddings=True) for sen in tqdm(df['user'].tolist())]

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1]['user']
    answer = row[1]['answer']
    types = row[1]['class']
    
    metadata = {
        "query": query,
        "answer": answer
    }
    ids.append(str(index))
    metadatas.append(metadata)
    
chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

100%|███████████████████████████████████████| 34549/34549 [13:08<00:00, 43.80it/s]
34549it [00:01, 29961.38it/s]
100%|█████████████████████████████████████████████| 34/34 [00:30<00:00,  1.12it/s]


In [11]:
query_embeddings = embedding_query("자꾸만 눈물이 나와", normalize_embeddings=True).tolist()
result = answers.query(
    query_embeddings=query_embeddings,
    n_results=1
)

result_df = pd.DataFrame(result['metadatas'][0])
result_df['distances'] = result['distances'][0]
result_df

,answer,query,distances
0,"요즘 많이 힘들었군요.. 제가 종종 하는 방법인데, 선생님의 마음을 한번 종이에 적...",계속 눈물이 나.,0.179987
